In [2]:
import os

try:
    GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
    if(GOOGLE_API_KEY):
        print("Key found")
    else:
        print("Key not found")
except Exception as e:
    print(f"Error:{e}")

Key found


In [3]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK components imported")


ADK components imported


In [5]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")


✅ Helper functions defined.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 1.1 Memory Agent

### 1.1.1 Initialize Memory
`InMemoryMemoryService` - Built in service  
`VertexAiMemoryBankService` - cloud Service

In [9]:
memory_service = InMemoryMemoryService()
session_service = InMemorySessionService()

### 1.1.2 Add memory to agent

In [8]:
#Define constants used throughout the notebook
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

print("✅ Agent created")

✅ Agent created


In [10]:
runner = Runner(
    agent = user_agent,
    app_name = APP_NAME,
    session_service = session_service,
    memory_service = memory_service,
)
print("✅ Runner created")

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Runner created


**Adding memory service to runner makes memory available to agent, but it doesnt use it auotmatically**

### 1.1.3 Ingest Session data into memory

In [21]:
# User tells agent about their favorite color
await run_session(
    runner,
    "My favorite color is blue-green. Can you write a Haiku about it?",
    "conversation-01",  # Session ID
)



### Session: conversation-01

User > My favorite color is blue-green. Can you write a Haiku about it?
Model: > Deep ocean's whisper,
Or jungle leaves after rain,
Calming, soft embrace.


In [26]:
session = await session_service.get_session(app_name = APP_NAME, user_id = USER_ID, session_id = "conversation-01")


print("Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text
        if(event.content and event.content.parts)
        else "No text"
    )
    print(f"{event.content.role}: {text}")

Session contains:
user: My favorite color is blue-green. Can you write a Haiku about it?
model: A splash of ocean,
Turquoise waves on emerald leaves,
Calm and deep beauty.
user: My favorite color is blue-green. Can you write a Haiku about it?
model: The sea's gentle hue,
Like summer leaves in the sun,
Peaceful, lovely shade.
user: My favorite color is red. Can you write a Haiku about it?
model: A rose's soft bloom,
Fire's fierce and warming glow,
Bold and vibrant life.
user: My favorite color is blue-green. Can you write a Haiku about it?
model: Deep ocean's whisper,
Or jungle leaves after rain,
Calming, soft embrace.


In [27]:
await memory_service.add_session_to_memory(session)
print("✅ Session data added to memory")

✅ Session data added to memory


### 1.1.4 Enable memory retrieval

In [28]:
# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        load_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("✅ Agent with load_memory tool created.")

✅ Agent with load_memory tool created.


In [29]:
# Create a new runner with the updated agent
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What is my favorite color?", "color-test")

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".



### Session: color-test

User > What is my favorite color?


Model: > I'm sorry, I cannot find any information about your favorite color in your memory. 



In [30]:
await run_session(runner, "My birthday is on March 15th.", "birthday-session-01")


### Session: birthday-session-01

User > My birthday is on March 15th.
Model: > Got it. Your birthday is March 15th.


In [31]:
# Manually save the session to memory
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

print("✅ Birthday session saved to memory!")

✅ Birthday session saved to memory!


In [32]:
await run_session(
    runner, "When is my birthday?", "birthday-session-02"  # Different session ID
)


### Session: birthday-session-02

User > When is my birthday?


Model: > Your birthday is on March 15th.


In [33]:
# Search for color preferences
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's favorite color?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 9 relevant memories

  [user]: My favorite color is blue-green. Can you write a Haiku about it?...
  [user]: My favorite color is blue-green. Can you write a Haiku about it?...
  [MemoryDemoAgent]: The sea's gentle hue,
Like summer leaves in the sun,
Peaceful, lovely shade....
  [user]: My favorite color is red. Can you write a Haiku about it?...
  [MemoryDemoAgent]: A rose's soft bloom,
Fire's fierce and warming glow,
Bold and vibrant life....
  [user]: My favorite color is blue-green. Can you write a Haiku about it?...
  [MemoryDemoAgent]: Deep ocean's whisper,
Or jungle leaves after rain,
Calming, soft embrace....
  [user]: My birthday is on March 15th....
  [MemoryDemoAgent]: Got it. Your birthday is March 15th....


### 1.1.5 Automatic Memort Storage

In [34]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

✅ Callback created.


In [35]:
# Agent with automatic memory saving
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

print("✅ Agent created with automatic memory saving!")

✅ Agent created with automatic memory saving!


In [36]:
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)

print("✅ Runner created.")

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Runner created.


In [37]:
await run_session(
    auto_runner,
    "I gifted a new toy to my nephew on his 1st birthday!",
    "auto-save-test",
)



### Session: auto-save-test

User > I gifted a new toy to my nephew on his 1st birthday!
Model: > That's wonderful! A first birthday is such a special milestone. I hope your nephew loved his new toy!


In [38]:
await run_session(
    auto_runner,
    "What did I gift my nephew?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


### Session: auto-save-test-2

User > What did I gift my nephew?
Model: > You gifted your nephew a new toy.
